<a href="https://colab.research.google.com/github/evanjholt/sentiment-tracker/blob/Version2/Reddit_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install praw

In [24]:
import praw
import re
from datetime import datetime
import pandas as pd # Useful for data analysis and quick table views

In [25]:
# Cell 3: Load Stock Tickers from CSV (using GitHub Raw Link)

# --- IMPORTANT: Replace this URL with the actual raw link you copied from GitHub ---
csv_url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies-financials/refs/heads/main/data/constituents.csv'

# --- IMPORTANT: Verify the column name in your chosen CSV that contains the tickers ---
ticker_column_name = 'Symbol' # Common for NASDAQ/NYSE lists. Could be 'Ticker', 'Stock', etc.

try:
    print(f"Attempting to load tickers from: {csv_url}")
    all_us_tickers_df = pd.read_csv(csv_url)

    # Basic data cleaning and validation for tickers
    all_us_tickers_df = all_us_tickers_df.dropna(subset=[ticker_column_name])
    all_us_tickers_df = all_us_tickers_df[all_us_tickers_df[ticker_column_name].apply(lambda x: isinstance(x, str))]

    raw_stock_tickers = all_us_tickers_df[ticker_column_name].unique().tolist() # Store as raw_stock_tickers temporarily

    # Further clean and normalize tickers (alphanumeric, uppercase, typical length)
    stock_tickers_processed = [t for t in raw_stock_tickers if t.isalnum() and 1 <= len(t) <= 5] # Tickers are usually 1-5 chars
    stock_tickers = [t.upper() for t in stock_tickers_processed] # Assign directly to stock_tickers

    print(f"Loaded {len(stock_tickers)} unique tickers from GitHub URL.")
    if len(stock_tickers) > 0:
        print(f"First 10 tickers: {stock_tickers[:10]}")
    else:
        print("No tickers remain after basic processing. Check your CSV.")

except Exception as e:
    print(f"ERROR: Could not load tickers from GitHub URL. Error: {e}")
    print(f"WARNING: Using a smaller, hardcoded list as a fallback for testing.")
    # Fallback list (this fallback will now include some ambiguous words, which will be handled by Cell 4)
    stock_tickers = ['GME', 'AMC', 'TSLA', 'AAPL', 'MSFT', 'UNH', 'NVDA', 'AMD', 'BB', 'CLOV', 'SOFI', 'RIVN', 'AMZN', 'NFLX', 'GOOG', 'META', 'SPY', 'QQQ', 'VIX']

Attempting to load tickers from: https://raw.githubusercontent.com/datasets/s-and-p-500-companies-financials/refs/heads/main/data/constituents.csv
Loaded 501 unique tickers from GitHub URL.
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [26]:
from google.colab import userdata

reddit = praw.Reddit(
    client_id=userdata.get('REDDIT_CLIENT_ID'),
    client_secret=userdata.get('REDDIT_CLIENT_SECRET'),
    user_agent=userdata.get('REDDIT_USER_AGENT')
)

In [28]:
# Cell 4: Define and Execute the Reddit Scraper Function

import time # Ensure 'time' module is imported if not already in Cell 2

# --- NEW: Define a specific list for tickers that ONLY count with a '$' prefix ---
# These are tickers that are also very common English words.
# We will ONLY count these if they are explicitly preceded by a '$'.
# YOU WILL NEED TO EXPAND THIS LIST AS YOU DISCOVER MORE PROBLEMATIC TICKERS
# by observing your scrape results.
AMBIGUOUS_TICKERS_REQUIRING_DOLLAR_SIGN = {
    "A", "AN", "AND", "ARE", "AS", "AT", "BE", "BECAUSE", "BUT", "BY", "FOR", "FROM", "HAS",
    "HE", "HER", "HIS", "HOW", "I", "IN", "IS", "IT", "ITS", "OF", "ON", "OR", "THAT", "THE",
    "THIS", "TO", "WAS", "WHAT", "WHEN", "WHERE", "WHO", "WHY", "WILL", "WITH", "YOU",
    "YOUR", "ME", "MY", "WE", "OUR", "YOURS", "HIM", "HERS", "THEY", "THEM", "THEIR", "MINE",
    "DO", "DOES", "DID", "NOT", "NO", "YES", "AM", "ISN", "AREN", "WASN", "WEREN", "HAVEN",
    "HASN", "HAD", "ABOUT", "ABOVE", "AFTER", "AGAIN", "ALL", "AMONG", "ANY", "BELOW",
    "BETWEEN", "BOTH", "EACH", "FEW", "MORE", "MOST", "OTHER", "SOME", "SUCH", "THAN",
    "TOO", "VERY", "S", "T", "CAN", "JUST", "DON", "SHOULD", "NOW", "ONLY", "EVEN", "MUCH",
    "MANY", "ONE", "TWO", "THREE", "FOUR", "FIVE", "SIX", "SEVEN", "EIGHT", "NINE", "TEN",
    "GOOD", "HIGH", "LOW", "BIG", "OLD", "NEW", "RUN", "BUY", "SELL", "HOLD", "READ", "OUT",
    "SEE", "FUN", "HUB", "MAP", "PINE", "RIDE", "SAP", "MOON", "QT", "ZAP", "UPL", "WOLF",
    "JOY", "KISS", "LUCK", "VIEW", "YUM", "DOG", "EAT", "FROG", "CAT", "PUMP", "DUMP", "SPY",
    "QQQ", "VIX", "AI", "META", "GOOG", "COIN", "SHOP", "SNOW", "ROKU", "PLTR", "MRNA",
    "CFO", "YOLO", "ON", "SO", "GO", "US", "BTG", "GURU", "NICE", "OPEN", "TOP", "BED", "BOX",
    "COAL", "DASH", "FLOW", "GAME", "GOLD", "GRAB", "IRON", "KITE", "LIVE", "MOVE", "NEST",
    "RACE", "RAIL", "RICE", "SALT", "SHOT", "SILK", "STAR", "TALK", "TRUE", "UNIT", "VINE",
    "WISH", "YARD", "DAY", "BRO", "TECH", "WELL", "KEY", "FAST", "COST", "B", "C", "D", "E",
    "F", "G", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "V", "W", "X", "Y", "Z",
    "AMP", "AXP", "BAC", "BLK", "BLL", "BWA", "CI", "CL", "CMG", "COF", "COP", "CSX", "DE",
    "DFS", "DLB", "DOW", "DUK", "ECL", "EOG", "ETN", "F", "FCX", "GE", "GM", "GOOGL", "GS",
    "HD", "IBM", "ICE", "INTC", "JPM", "KO", "LEN", "LLY", "LMT", "MA", "MAR", "MCD", "MCO",
    "MMM", "MO", "MS", "MU", "NEE", "NKE", "NOC", "OKE", "ORLY", "PEP", "PFE", "PG", "PH",
    "PM", "PNC", "POOL", "PPG", "PGR", "QCOM", "RCL", "RL", "ROST", "SBUX", "SCHW", "SHW",
    "SLB", "SPG", "SRE", "STZ", "SYK", "TAP", "TGT", "TJX", "TRV", "TXN", "UDR", "UPS",
    "USB", "V", "VRTX", "VZ", "WBA", "WDC", "WFC", "WMB", "WM", "XOM", "ZTS"
  "BK", "BALL","GEN","DTE","FICO","PPL","REGN","HES","MET",'IFF', "DD",'DECK','keys'

}
# Ensure all entries are uppercase for consistent matching
AMBIGUOUS_TICKERS_REQUIRING_DOLLAR_SIGN = {word.upper() for word in AMBIGUOUS_TICKERS_REQUIRING_DOLLAR_SIGN}


def find_stock_mentions(text):
    """
    Identifies stock tickers within a given text based on ApeWisdom-like rules:
    1. Dollar sign prefixed tickers (e.g., $aapl, $AAPL) are always counted.
    2. Specific ambiguous tickers (defined in AMBIGUOUS_TICKERS_REQUIRING_DOLLAR_SIGN)
       ONLY count if they have a '$' prefix.
    3. Other all-uppercase tickers (not in the ambiguous list) are counted if they appear as whole words.
    """
    mentions = set()
    # Convert text to upper once for efficient case-insensitive word matching,
    # but keep the original text for dollar sign matching as '$' might not be upper.
    text_upper = text.upper()
    words_in_text_upper = re.findall(r'\b\w+\b', text_upper) # Extract all words from the upper-cased text

    for ticker in stock_tickers: # 'stock_tickers' is populated from Cell 3
        ticker_upper = ticker.upper() # Ensure ticker is uppercase for comparison

        # Rule 1: Always check for Dollar Sign prefix first (highest confidence)
        # Use case-insensitive matching for the ticker part, but require '$'
        if re.search(r'\$' + re.escape(ticker) + r'\b', text, re.IGNORECASE):
            mentions.add(ticker_upper)
            continue # Move to the next ticker as we've found a strong match for this one

        # Rule 2: If it's an ambiguous ticker (like ON, CAN, CFO) AND it did NOT have a dollar sign, then skip it.
        if ticker_upper in AMBIGUOUS_TICKERS_REQUIRING_DOLLAR_SIGN:
            continue # This ticker is ambiguous and lacked a '$', so we ignore it as a mention.

        # Rule 3: For all other non-ambiguous tickers, check for an all-uppercase word match.
        # This catches tickers like "AAPL", "MSFT" when they appear as standalone all-caps words.
        if ticker_upper in words_in_text_upper:
            # We already handled the '$' case and ambiguous tickers that need '$'.
            # So, if we reach here, it's a non-ambiguous ticker matched as an all-caps word.
            mentions.add(ticker_upper)

    return list(mentions)


def scrape_reddit_mentions():
    """
    Scrapes a limited number of new submissions and their comments
    from specified subreddits for stock mentions.
    Returns a dictionary where keys are stock tickers and values are lists of mention details.
    """
    stock_data = {}
    # You can start with one or two subreddits for faster testing, then add more.
    subreddits_to_monitor = ['stocks', 'investing', 'wallstreetbets'] # Add/remove as desired

    print("Starting Reddit scraping job...")
    print(f"Monitoring {len(stock_tickers)} tickers across {len(subreddits_to_monitor)} subreddits.")

    # --- Configuration for Scraping Limit and Pauses ---
    # Adjust 'scrape_limit_per_subreddit' based on how much data you want and your observed speed.
    # A value of 50-100 per subreddit is a good balance for testing in Colab.
    scrape_limit_per_subreddit = 100 # Number of 'new' submissions to fetch per subreddit

    # Pause duration: higher values mean slower but safer scraping.
    pause_between_submissions = 1.5 # seconds to pause after a few submissions
    pause_between_subreddits = 7 # seconds to pause after processing an entire subreddit

    for subreddit_name in subreddits_to_monitor:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"\n--- Scraping r/{subreddit_name} (fetching top {scrape_limit_per_subreddit} new posts)... ---")

            # Fetch new submissions
            for i, submission in enumerate(subreddit.new(limit=scrape_limit_per_subreddit)):
                # Introduce a small pause periodically to respect Reddit API limits
                if i > 0 and i % 5 == 0: # Pause every 5 submissions
                    print(f"  Pausing {pause_between_submissions}s after {i} submissions in r/{subreddit_name}...")
                    time.sleep(pause_between_submissions)

                submission_text = submission.title + " " + submission.selftext # Combine title and body
                mentions_in_submission = find_stock_mentions(submission_text)

                if mentions_in_submission:
                    # print(f"    Found mentions in submission '{submission.title[:50]}...' for: {mentions_in_submission}") # Uncomment for detailed debug
                    for mention in mentions_in_submission:
                        if mention not in stock_data:
                            stock_data[mention] = []
                        stock_data[mention].append({
                            'type': 'submission',
                            'subreddit': subreddit_name,
                            'id': submission.id,
                            'title': submission.title,
                            'url': submission.url,
                            'score': submission.score,
                            'created_utc': submission.created_utc, # Unix timestamp
                            'timestamp': datetime.fromtimestamp(submission.created_utc).isoformat() # ISO 8601 string
                        })

                # Process comments for each submission
                # replace_more(limit=0) fetches only directly available comments,
                # avoiding extra API calls for "More Comments" links which is good for testing.
                submission.comments.replace_more(limit=0)
                for comment in submission.comments.list():
                    mentions_in_comment = find_stock_mentions(comment.body)
                    if mentions_in_comment:
                        # print(f"      Found mentions in comment '{comment.body[:50]}...' for: {mentions_in_comment}") # Uncomment for detailed debug
                        for mention in mentions_in_comment:
                            if mention not in stock_data:
                                stock_data[mention] = []
                            stock_data[mention].append({
                                'type': 'comment',
                                'subreddit': subreddit_name,
                                'id': comment.id,
                                'submission_id': submission.id, # Link comment to its parent submission
                                'comment_body': comment.body,
                                'url': f"https://reddit.com{comment.permalink}",
                                'score': comment.score,
                                'created_utc': comment.created_utc,
                                'timestamp': datetime.fromtimestamp(comment.created_utc).isoformat()
                            })
        except Exception as e:
            print(f"ERROR: Failed to scrape r/{subreddit_name}. Error: {e}")

        # Pause after finishing an entire subreddit, gives a larger break before the next subreddit
        print(f"  Finished r/{subreddit_name}. Pausing for {pause_between_subreddits}s before next subreddit...")
        time.sleep(pause_between_subreddits)

    print("\nReddit scraping job finished.")
    return stock_data

# --- Execute the scraper function and store results ---
test_mentions = scrape_reddit_mentions()

print("\n--- Scrape Result Summary ---\n")
if test_mentions:
    total_mentions_found = sum(len(data) for data in test_mentions.values())
    print(f"Found {total_mentions_found} mentions for {len(test_mentions)} unique stocks.")
    # Show top 10 mentioned stocks by raw count
    sorted_stocks = sorted(test_mentions.items(), key=lambda item: len(item[1]), reverse=True)
    print("\nTop 10 Most Mentioned Stocks (Raw Count):")
    for stock, data in sorted_stocks[:10]:
        print(f"  - {stock}: {len(data)} mentions")
else:
    print("No stock mentions found in this scrape. Try increasing scrape_limit_per_subreddit or check your ticker lists.")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Starting Reddit scraping job...
Monitoring 501 tickers across 3 subreddits.

--- Scraping r/stocks (fetching top 100 new posts)... ---


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

  Pausing 1.5s after 5 submissions in r/stocks...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

...

  Pausing 1.5s after 90 submissions in r/wallstreetbets...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

  Pausing 1.5s after 95 submissions in r/wallstreetbets...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

  Finished r/wallstreetbets. Pausing for 7s before next subreddit...

Reddit scraping job finished.

--- Scrape Result Summary ---

Found 848 mentions for 120 unique stocks.

Top 10 Most Mentioned Stocks (Raw Count):
  - UNH: 270 mentions
  - NVDA: 93 mentions
  - TSLA: 88 mentions
  - MSFT: 31 mentions
  - AMZN: 30 mentions
  - AAPL: 23 mentions
  - UBER: 19 mentions
  - AMD: 16 mentions
  - NFLX: 12 mentions
  - FOX: 10 mentions


In [33]:
# Cell 5: Analyze Scraped Data with Pandas and Generate Ranking Table

import pandas as pd # Ensure pandas is imported, likely already done in Cell 2

# Flatten the dictionary of mentions into a list of dictionaries
all_mentions_list = []
for ticker, mentions in test_mentions.items():
    for mention_detail in mentions:
        mention_detail['ticker'] = ticker # Add the stock ticker to each mention record
        all_mentions_list.append(mention_detail)

if all_mentions_list:
    # Create a Pandas DataFrame from the list of mentions
    df = pd.DataFrame(all_mentions_list)

    print("\n--- DataFrame Head (First 5 rows) ---")
    print(df.head()) # Shows the first 5 rows of your data

    print("\n--- Basic DataFrame Info ---\n")
    df.info() # Provides a summary of the DataFrame structure and data types

    print("\n--- Stock Mention Counts (Overall Ranking) ---\n")
    # Count how many times each stock ticker was mentioned
    mention_counts = df['ticker'].value_counts().reset_index()
    mention_counts.columns = ['Ticker', 'Mentions']
    print(mention_counts.head(15)) # Show top 15 by count

    print("\n--- Mentions by Type (Submission vs. Comment) ---\n")
    print(df['type'].value_counts())

    print("\n--- Subreddit Mention Counts ---\n")
    print(df['subreddit'].value_counts())

    print("\n--- Top 10 Submissions by Score (Most Engaged Posts) ---\n")
    # Filter for submissions and sort by 'score' (upvotes)
    submissions_df = df[df['type'] == 'submission'].sort_values(by='score', ascending=False)
    if not submissions_df.empty:
        # Display relevant columns for top submissions
        print(submissions_df.head(10)[['ticker', 'title', 'score', 'url', 'timestamp']])
    else:
        print("No submission data found to show top posts.")

    print("\n--- Top 10 Comments by Score (Most Engaged Comments) ---\n")
    # Filter for comments and sort by 'score'
    comments_df = df[df['type'] == 'comment'].sort_values(by='score', ascending=False)
    if not comments_df.empty:
        # Display relevant columns for top comments
        print(comments_df.head(10)[['ticker', 'comment_body', 'score', 'url', 'timestamp']])
    else:
        print("No comment data found to show top comments.")

    # --- Prepare data for ranking table ---
    # Group by ticker to sum mentions and upvotes
    stock_summary = df.groupby('ticker').agg(
        Mentions=('ticker', 'size'), # Count mentions
        Upvotes=('score', 'sum')     # Sum scores for total upvotes
    ).reset_index()
    stock_summary.columns = ['Ticker', 'Mentions', 'Upvotes']

    # Sort by mentions to determine rank
    stock_summary = stock_summary.sort_values(by='Mentions', ascending=False).reset_index(drop=True)
    stock_summary['Rank'] = stock_summary.index + 1 # Add a rank column, starting from 1

    # Select the columns for the final display table and set their order
    final_ranking_table = stock_summary[['Rank', 'Ticker', 'Mentions', 'Upvotes']]

    # Select top N for display (e.g., top 20 or more)
    top_n_stocks_for_table = 50 # You can adjust this number
    display_table = final_ranking_table.head(top_n_stocks_for_table)

    print(f"\n--- Top {top_n_stocks_for_table} Stocks Ranked by Mentions (Table) ---\n")
    # Print the table with desired column headers and format
    print(display_table.to_string(index=False)) # .to_string(index=False) prevents printing the DataFrame index


else:
    print("\nNo mentions were found in the DataFrame to analyze or rank.")


--- DataFrame Head (First 5 rows) ---
         type subreddit       id                     title  \
0  submission    stocks  1ksxlx8  Why Shorting Never Works   
1     comment    stocks  mtp6fg6                       NaN   
2     comment    stocks  mtphi31                       NaN   
3     comment    stocks  mtncreb                       NaN   
4     comment    stocks  mtnfhjo                       NaN   

                                                 url  score   created_utc  \
0  https://www.reddit.com/r/stocks/comments/1ksxl...      0  1.747937e+09   
1  https://reddit.com/r/stocks/comments/1ksu73r/k...      3  1.747938e+09   
2  https://reddit.com/r/stocks/comments/1ksu73r/k...      3  1.747942e+09   
3  https://reddit.com/r/stocks/comments/1kspujg/s...    103  1.747919e+09   
4  https://reddit.com/r/stocks/comments/1kspujg/s...      1  1.747920e+09   

             timestamp ticker submission_id  \
0  2025-05-22T18:08:31   TSLA           NaN   
1  2025-05-22T18:26:00   TSLA  